In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Introduction — Job-Seeking Multi-Agent AI System**

This notebook demonstrates the design and implementation of a multi-agent AI system for intelligent job-seeking support. Built using Google’s Agent Development Kit (ADK) and Gemini models, the system automates several key stages in the job application process, including:

Analysing a job description for required skills and ATS keywordsRevising and tailoring a candidate’s CV for the roleGenerating a professional cover letterResearching company context and recent newsSupporting interview preparation

Rather than relying on a single monolithic agent, this system follows a multi-agent architecture, where each agent is specialized for a specific task (e.g., job analysis, CV rewriting, research, or coaching). These agents are coordinated by a root agent that ensures all steps are executed in the correct sequence.

The overall objective of this notebook is to demonstrate how agentic AI can be applied to a real-world use case: improving efficiency, relevance, and personalization in the job-application workflow.

This system can be extended and adapted to other career-related scenarios, such as internship matching, career advising, or recruitment analytics.

This block retrieves the Google API key from Kaggle’s secrets manager and stores it as an environment variable. This key is required for authenticating calls to the Gemini model used by the agents.

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


This block imports the core components of the Google Agent Development Kit (ADK), including:

Agent and specialized agent types (Sequential, Parallel, Loop)
Gemini as the underlying LLM
google_search as an external information tool
InMemoryRunner for executing and testing agents

In [3]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.tools import AgentTool, FunctionTool, google_search

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


This function extracts and displays the tool-generated Python code and its results from the agent’s response object. It is useful for debugging and transparency when agents use a code executor.

In [4]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


This block defines HTTP retry rules in the event of API failures, such as:

Rate limits (429)

Server errors (500+)

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

 ## **Agent Systems**

 * job research agent - provide similar job openings
 * job analyst agent - conduct job analysis for the job
 * CV writer agent - revised the cv with the keywords identified
 * CL writer agent - draft a cover letter based on revised cv and job description
 * Coach agent - proposed some interview questions and mock answers

In [6]:
#job research agent

research_agent = Agent(
    name="research_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are a job search agent.

You MUST use the google_search tool.

Based on the job description in the conversation, find 3 real job postings that are similar to this role:
- Titles: Data & Insights Analyst, Data Analyst, CX Analyst, Customer Insights Analyst, or very similar.
- Location: Toronto, ON OR Remote roles open to candidates in Canada.

For EACH job, return:
- Job Title
- Company
- Location
- URL

Output format:
- A markdown bullet list.
- Each bullet in the format:
  - Job Title – Company (Location) — URL

Rules:
- Use google_search to find current postings (do NOT invent companies).
- Return ONLY the bullet list. No extra commentary.
""",
    tools=[google_search],
    output_key="similar_jobs",
)

print("Research_agent updated with output_key='similar_jobs'")


#job analyst agent

job_analyst_agent = Agent(
    name="job_analyst_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are an ATS-style job analysis agent.

You will receive a single message that contains BOTH:
- A job description
- A candidate CV

Your tasks:
1. Identify and list:
   - Hard Skills & Technologies
   - Soft Skills
   - Key Qualifications & Education
2. Extract the Top 15 ATS Keywords (comma-separated).
3. Estimate an ATS match score (0–100%) for how well this CV fits the job,
   based on keyword overlap, experience alignment, and responsibilities.

Output format (markdown):

### Hard Skills & Technologies
- ...

### Soft Skills
- ...

### Key Qualifications & Education
- ...

### Top 15 ATS Keywords
keyword1, keyword2, ...

### ATS Match Score
XX%

Return ONLY this analysis in markdown.
""",
    output_key="keywords",   # <- we’ll print this block, including the ATS score
)

print("job_analyst_agent updated with ATS scoring")


# CV Writer Agent

cvwriter_agent = Agent(
    name="cvwriter_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are an expert CV writer.

You will receive ONE message that contains:
- A job description after the line: "========= JOB DESCRIPTION ========="
- A current CV after the line: "========= CURRENT CV ========="

Your job:
1. Read the job description and identify the key skills, responsibilities, and ATS keywords.
2. Rewrite the CV so it is clearly tailored to this role.
3. Reorder and rephrase experience bullets to highlight the most relevant achievements.
4. Use metrics where possible (e.g., "15+ campaigns", "20% improvement", etc.).
5. Keep the format clean, professional, and ATS-friendly.

Very important:
- Assume the job description and CV are already provided in the same message.
- Do NOT ask the user to paste anything.
- Do NOT explain what you are going to do.
- Do NOT repeat the job description.
- Do NOT repeat the original CV.

Return ONLY the final revised CV.
""",
    output_key="revisedcv",
)
print("cvwriter_agent updated.")


#Cover letter writing agent

clwriter_agent = Agent(
    name="clwriter_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are a professional cover letter writer.

You will receive ONE message that contains:
- A job description
- A current CV (candidate profile)

Your job:
1. Write a tailored cover letter for this specific role and company.
2. Use a clear business letter format with:
   - Candidate name and contact info at the top (you can infer from the CV).
   - Date
   - Greeting ("Dear Hiring Manager," if no name is given)
   - 3–4 short paragraphs:
     * Opening: role applied for and a 1–2 sentence value proposition.
     * Middle: 1–2 paragraphs linking the candidate’s experience to the job requirements with concrete examples.
     * Closing: enthusiasm + call to action.
   - Professional closing and name.

Very important:
- Base the content on the job description and CV provided in the message.
- Do NOT ask for more information.
- Do NOT explain what you are doing.
- Do NOT repeat the full job description or full CV.

Return ONLY the final cover letter text.
""",
    output_key="coverletter",
)
print("clwriter_agent updated.")



Research_agent updated with output_key='similar_jobs'
job_analyst_agent updated with ATS scoring
cvwriter_agent updated.
clwriter_agent updated.


In [7]:
#Interview Coach Agent

coach_agent = Agent(
    name="coach_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
You are an interview coach.

You will receive a message that contains:
- The job description
- The candidate's CV (or revised CV)

Your job is to produce an interview preparation guide with THREE sections, all tailored to THIS role and THIS candidate:

## Behavioural / Fit Questions
- 5–7 behavioural or culture-fit questions.
- For each question, provide a strong sample answer in the first person ("I ...") that fits the candidate’s profile.

## Technical / Analytics Questions
- 5–7 technical / analytics questions related to:
  - SQL / querying data
  - Dashboards (Power BI)
  - Customer Experience metrics (NPS, CSAT, CES)
  - A/B testing and experiment design
  - Survey design and analysis
- For each question, provide a strong sample answer tailored to the candidate.

## Questions to Ask the Employer
- 5 thoughtful questions the candidate can ask the interviewer about:
  - The role
  - The data / tools
  - The team and stakeholders
  - Expectations and success metrics

Important rules:
- Base everything on THIS specific job and candidate.
- Do NOT ask the user for more information.
- Do NOT explain what you are doing.
- Do NOT repeat the full job description or CV.

Return ONLY the interview preparation guide in markdown.
""",
    output_key="interview_guide",
)

print("Coach_agent updated with output_key='interview_guide'")


Coach_agent updated with output_key='interview_guide'


## **Core inputs for the system**

* The job ad defines the target requirements
* The CV is the raw material for transformation

In [8]:
job_ad_text = """Job Title: Data & Insights Analyst – Customer Experience

Company: Nova Retail Solutions Inc.
Location: Toronto, ON (Hybrid – 3 days in office)
Salary Range: $70,000 – $90,000 per year

About the Company
Nova Retail Solutions is a fast-growing omni-channel retail analytics company that helps national brands understand customer behaviour across online and in-store environments. We use data, AI, and customer research to optimize experiences and increase lifetime value.

About the Role

We are seeking a Data & Insights Analyst – Customer Experience to join our Customer Intelligence team. In this role, you will support the analysis, reporting, and interpretation of customer experience (CX) data and deliver actionable insights to internal stakeholders.

This is an ideal opportunity for a detail-oriented analyst who enjoys working with data, generating stories, and influencing business decisions.

Key Responsibilities

• Analyze customer data from multiple sources (survey results, CRM, web analytics, loyalty programs)
• Design and manage customer surveys to measure sentiment, satisfaction, and NPS
• Build weekly and monthly dashboards using Power BI
• Conduct quantitative and qualitative analysis on customer feedback
• Identify patterns, trends, and opportunities to improve customer journeys
• Present findings and recommendations to marketing, product, and leadership teams
• Support A/B testing initiatives and campaign measurement

Required Qualifications

• Bachelor’s degree in Business Analytics, Marketing, Statistics, or related field
• 2+ years of experience in data analysis or market research
• Strong proficiency in Excel (advanced formulas, pivot tables, VLOOKUP)
• Experience with Power BI or Tableau
• Basic knowledge of SQL for data querying
• Understanding of customer experience metrics (NPS, CSAT, CES)
• Strong written and verbal communication skills

Preferred Qualifications

• Experience in retail, e-commerce, or consumer goods industries
• Familiarity with Python or R for analysis
• Knowledge of survey platforms (Qualtrics, SurveyMonkey)
• Experience working with CRM datasets

Key Skills

Data Analysis, Customer Experience (CX), Power BI, Excel, SQL, Survey Design, A/B Testing, Reporting, Stakeholder Communication, Data Visualization, Machine Learning (basic), Marketing Analytics

Why Join Us?

• Work with a collaborative and innovative team
• Opportunity for professional growth and learning
• Competitive salary and benefits
• Hybrid work environment
• High-impact projects supporting top retail brands

How to Apply:
Submit your resume and cover letter to careers@novaretail.ai
"""

base_cv_text = """Name: Alex Thompson
Location: Toronto, ON
Email: alex.thompson@email.com
Phone: 647-555-0198
LinkedIn: linkedin.com/in/alexthompson

Professional Summary

Aspiring data analyst with strong analytical skills and a background in business and marketing. Experienced in handling survey data, building dashboards, and supporting decision-making with data-driven insights. Seeking a growth opportunity in customer experience analytics.

Education

Bachelor of Commerce (Marketing & Analytics)
University of Toronto, 2021

Technical Skills

• Microsoft Excel (advanced)
• Power BI (intermediate)
• SQL (basic)
• Python (beginner)
• SPSS
• Google Analytics
• SurveyMonkey, Qualtrics

Work Experience

Marketing Data Assistant
BrightPath Marketing Agency
Jan 2023 – Present

• Supported data analysis for 15+ client campaigns
• Built Power BI dashboards to track customer engagement
• Cleaned and organized survey and CRM data
• Prepared weekly reports for account managers
• Helped conduct A/B testing on email campaigns

Customer Service Associate
Urban Essentials Retail
Jun 2021 – Dec 2022

• Assisted 50+ customers daily and resolved inquiries
• Collected customer feedback and suggestions
• Maintained high customer satisfaction ratings
• Identified recurring issues and reported to management

Academic Project – Customer Satisfaction Study

• Designed and distributed survey to 250 participants
• Analyzed responses using SPSS and Excel
• Presented insights on factors affecting satisfaction

Certifications

• Google Data Analytics Certificate (Coursera)
• Power BI Fundamentals (Microsoft Learn)

Soft Skills

Analytical Thinking
Communication
Attention to Detail
Problem Solving
Teamwork
Customer Focus
"""


## **Individual Agent Execution Pipeline**

**This function allows the system to:**

* Run each agent independently
* Extract its output from the state
* Store results in separate variables


**Execution Order**

* ATS & keyword analysis
* CV revision
* Cover letter generation
* Interview guide
* Similar job discovery

Each is handled by its specialised agent, showing clean modular design.

In [9]:
from google.adk.runners import InMemoryRunner

# Shared context: job ad + CV
context_prompt = f"""
========= JOB DESCRIPTION =========
{job_ad_text}

========= CURRENT CV =========
{base_cv_text}
"""

def extract_state_from_events(events):
    final_state = {}
    if not isinstance(events, (list, tuple)):
        events = [events]
    for e in events:
        actions = getattr(e, "actions", None)
        if actions and getattr(actions, "state_delta", None):
            final_state.update(actions.state_delta)
    return final_state

# Helper to run a single agent and get its state
async def run_agent(agent, prompt):
    runner = InMemoryRunner(agent=agent)
    # quiet=True stops printing "User > ..." and agent traces
    events = await runner.run_debug(prompt, quiet=True, verbose=False)
    return extract_state_from_events(events)


# 1) ATS + keywords (job_analyst_agent)
keywords_state = await run_agent(job_analyst_agent, context_prompt)
keywords_text = keywords_state.get("keywords", "No keywords / ATS analysis found.")

# 2) Revised CV (cvwriter_agent)
cv_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Rewrite the CV so it is clearly tailored to this role.
- Use the job description to emphasise relevant skills and experience.
- Use metrics where possible.
- Keep the format clean and ATS-friendly.

Return ONLY the revised CV.

{context_prompt}
"""
cv_state = await run_agent(cvwriter_agent, cv_prompt)
revised_cv_text = cv_state.get("revisedcv", "No revised CV found.")

# 3) Cover letter (clwriter_agent)
cl_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Write a professional cover letter for this specific role and candidate.
- Use a clear business letter format.
- Base the content on the job description and the candidate's profile.

Return ONLY the final cover letter.

{context_prompt}
"""
cl_state = await run_agent(clwriter_agent, cl_prompt)
cover_letter_text = cl_state.get("coverletter", "No cover letter found.")

# 4) Interview prep (coach_agent)
coach_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Generate an interview preparation guide for this role and candidate:
  - 5–7 Behavioural / Fit questions + sample answers
  - 5–7 Technical / Analytics questions + sample answers
  - 5 smart questions the candidate should ask the employer

Return ONLY the interview preparation guide in markdown.

{context_prompt}
"""
coach_state = await run_agent(coach_agent, coach_prompt)
interview_guide_text = coach_state.get("interview_guide", "No interview guide found.")

# 5) Similar jobs (research_agent) – job ad is enough
jobs_prompt = f"""
Here is a job description for a Data & Insights Analyst – Customer Experience role:

{job_ad_text}

Use google_search to find 3 similar jobs:
- Similar title (Data & Insights Analyst, CX Analyst, Customer Insights Analyst, etc.)
- Located in Toronto, ON or Remote (Canada)
Return ONLY a markdown bullet list:
- Job Title – Company (Location) — URL
"""
jobs_state = await run_agent(research_agent, jobs_prompt)
similar_jobs_text = jobs_state.get("similar_jobs", "No similar jobs found.")

# ---------- Print everything nicely ----------

print("\n===== ATS / KEYWORDS =====\n")
print(keywords_text)

print("\n===== REVISED CV =====\n")
print(revised_cv_text)

print("\n===== COVER LETTER =====\n")
print(cover_letter_text)

print("\n===== INTERVIEW PREP (COACH AGENT) =====\n")
print(interview_guide_text)

print("\n===== SIMILAR JOBS (GOOGLE SEARCH) =====\n")
print(similar_jobs_text)



===== ATS / KEYWORDS =====

### Hard Skills & Technologies
- Microsoft Excel (advanced formulas, pivot tables, VLOOKUP)
- Power BI
- SQL (basic)
- Python (beginner)
- Google Analytics
- SurveyMonkey
- Qualtrics
- SPSS
- A/B Testing
- Data Analysis
- Reporting
- Data Visualization

### Soft Skills
- Analytical Thinking
- Communication (written and verbal)
- Attention to Detail
- Problem Solving
- Teamwork
- Customer Focus
- Stakeholder Communication (implied through reporting to account managers and presenting insights)

### Key Qualifications & Education
- Bachelor of Commerce (Marketing & Analytics)
- 2+ years of experience in data analysis or market research (Jan 2023 - Present as Marketing Data Assistant + Academic Project)
- Understanding of customer experience metrics (implied through survey analysis, customer feedback collection)

### Top 15 ATS Keywords
Data Analysis, Customer Experience, CX, Power BI, Excel, SQL, Survey Design, A/B Testing, Reporting, Data Visualization, Marke

## **Interactive Assistant Mode**

**Purpose**

The final section converts the notebook into an interactive AI assistant, enabling the user to:

* Paste any job ad
* Paste any CV

Choose desired tasks:
* CV
* Cover letter
* Interview guide
* Similar jobs
* All

In [10]:
def extract_state_from_events(events):
    final_state = {}
    if not isinstance(events, (list, tuple)):
        events = [events]
    for e in events:
        actions = getattr(e, "actions", None)
        if actions and getattr(actions, "state_delta", None):
            final_state.update(actions.state_delta)
    return final_state

async def run_agent(agent, prompt):
    runner = InMemoryRunner(agent=agent)
    events = await runner.run_debug(prompt, quiet=True, verbose=False)
    return extract_state_from_events(events)


In [11]:
# ==== 1. Collect user inputs interactively ====

print("🧾 Job Application Assistant")
print("I'll help you with ATS analysis, CV tailoring, cover letter, interview prep, and similar jobs.\n")

# In a notebook, easiest is to paste the text into multi-line string cells,
# but if you want to input via console, use this pattern with 'END' markers.

def read_multiline(prompt_label):
    print(f"\nPaste your {prompt_label} below. Type a single line 'END' when you are done:\n")
    lines = []
    while True:
        line = input()
        if line.strip() == "END":
            break
        lines.append(line)
    return "\n".join(lines)

job_ad_text = read_multiline("JOB AD")
base_cv_text = read_multiline("CURRENT CV")

print("\nWhat would you like me to generate?")
print("Options: cv, cover_letter, interview guide, jobs, all")
tasks_input = input("Enter one or more, separated by commas (e.g. cv, cover_letter, interview, jobs): ")

# Normalize tasks
selected_tasks = {t.strip().lower() for t in tasks_input.split(",") if t.strip()}
if "all" in selected_tasks:
    selected_tasks = {"cv", "cover_letter", "interview", "jobs"}

print(f"\nYou selected: {', '.join(sorted(selected_tasks)) or 'nothing (default: ATS only)'}")


🧾 Job Application Assistant
I'll help you with ATS analysis, CV tailoring, cover letter, interview prep, and similar jobs.


Paste your JOB AD below. Type a single line 'END' when you are done:



StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
# ==== 2. Run selected agents and print outputs ====

context_prompt = f"""
========= JOB DESCRIPTION =========
{job_ad_text}

========= CURRENT CV =========
{base_cv_text}
"""

# Always do ATS / keywords (cheap + useful)
keywords_state = await run_agent(job_analyst_agent, context_prompt)
keywords_text = keywords_state.get("keywords", "No keywords / ATS analysis found.")

revised_cv_text = None
cover_letter_text = None
interview_guide_text = None
similar_jobs_text = None

# CV rewrite
if "cv" in selected_tasks:
    cv_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Rewrite the CV so it is clearly tailored to this role.
- Use the job description to emphasise relevant skills and experience.
- Use metrics where possible.
- Keep the format clean and ATS-friendly.

Return ONLY the revised CV.

{context_prompt}
"""
    cv_state = await run_agent(cvwriter_agent, cv_prompt)
    revised_cv_text = cv_state.get("revisedcv", "No revised CV found.")

# Cover letter
if "cover_letter" in selected_tasks:
    cl_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Write a professional cover letter for this specific role and candidate.
- Use a clear business letter format.
- Base the content on the job description and the candidate's profile.

Return ONLY the final cover letter.

{context_prompt}
"""
    cl_state = await run_agent(clwriter_agent, cl_prompt)
    cover_letter_text = cl_state.get("coverletter", "No cover letter found.")

# Interview prep guide
if "interview" in selected_tasks:
    coach_prompt = f"""
You will receive a job description and a current CV.

Your task:
- Generate an interview preparation guide for this role and candidate:
  - 5–7 Behavioural / Fit questions + sample answers
  - 5–7 Technical / Analytics questions + sample answers
  - 5 smart questions the candidate should ask the employer

Return ONLY the interview preparation guide in markdown.

{context_prompt}
"""
    coach_state = await run_agent(coach_agent, coach_prompt)
    interview_guide_text = coach_state.get("interview_guide", "No interview guide found.")

# Similar jobs
if "jobs" in selected_tasks:
    jobs_prompt = f"""
Here is a job description for a Data & Insights Analyst – Customer Experience role:

{job_ad_text}

Use google_search to find 3 similar jobs:
- Similar title (Data & Insights Analyst, CX Analyst, Customer Insights Analyst, etc.)
- Located in Toronto, ON or Remote (Canada)
Return ONLY a markdown bullet list:
- Job Title – Company (Location) — URL
"""
    jobs_state = await run_agent(research_agent, jobs_prompt)
    similar_jobs_text = jobs_state.get("similar_jobs", "No similar jobs found.")

# ==== 3. Print everything nicely ====

print("\n===== ATS / KEYWORDS =====\n")
print(keywords_text)

if "cv" in selected_tasks:
    print("\n===== REVISED CV =====\n")
    print(revised_cv_text)

if "cover_letter" in selected_tasks:
    print("\n===== COVER LETTER =====\n")
    print(cover_letter_text)

if "interview" in selected_tasks:
    print("\n===== INTERVIEW PREP (COACH AGENT) =====\n")
    print(interview_guide_text)

if "jobs" in selected_tasks:
    print("\n===== SIMILAR JOBS (GOOGLE SEARCH) =====\n")
    print(similar_jobs_text)
